In [7]:
pip install numpy==1.23.5



SyntaxError: invalid syntax (3136643634.py, line 1)

In [9]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Feature extraction
def extract_features_from_file(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfcc.T, axis=0)  # Mean across time axis

# Load data from folder
def load_data_from_folders(base_folder, labels):
    data = []
    target = []
    for label in labels:
        folder = os.path.join(base_folder, label)
        for filename in os.listdir(folder):
            if filename.endswith(".wav"):
                file_path = os.path.join(folder, filename)
                features = extract_features_from_file(file_path)
                data.append(features)
                target.append(label)
    return np.array(data), np.array(target)

# Set paths and labels
base_folder = '/Users/suarezcruzrosy/Downloads/animals'
labels = ['dog', 'cat', 'bird']

# Load data
X, y = load_data_from_folders(base_folder, labels)

# Encode labels to integers
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# KNN with GridSearchCV
param_grid = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best Parameters
print("Best Parameters:", grid_search.best_params_)

# Model Evaluation
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")


ImportError: Numba needs NumPy 1.22 or greater. Got NumPy 1.21.